In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import time

In [2]:
# GPU desteği var mı kontrol et
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Metal GPU backend
    print("MPS (GPU) kullanılacak.")
else:
    device = torch.device("cpu")
    print("GPU desteklenmiyor, CPU kullanılacak.")

MPS (GPU) kullanılacak.


In [35]:
def read_images(path, num_img):
    array = np.zeros((num_img, 64*32))
    i = 0
    for img in os.listdir(path):
        img_path = path + '/' + img
        img = Image.open(img_path, mode= 'r')
        data = np.asanyarray(img, dtype= 'uint8')
        data = data.flatten()
        array[i,:] = data
        i += 1
    return array

<h3>Negatives

In [36]:
train_negative_path = r'Data/LSIFIR/Classification/Train/neg'
num_train_negative_img = 43390

In [37]:
train_negative_array = read_images(train_negative_path, num_train_negative_img)

In [38]:
x_train_negative_tensor = torch.from_numpy(train_negative_array)

In [39]:
y_train_negative_tensor = torch.zeros(num_train_negative_img, dtype= torch.long)

<h3>Positives

In [41]:
train_positive_path = r'Data/LSIFIR/Classification/Train/pos'
num_train_positive_img = 10208

In [42]:
train_positive_array = read_images(train_positive_path, num_train_positive_img)

In [43]:
x_train_positive_tensor = torch.from_numpy(train_positive_array)

In [44]:
y_train_positive_tensor = torch.ones(num_train_positive_img, dtype= torch.long)

<h3>Concat Train

In [45]:
x_train = torch.cat((x_train_negative_tensor,x_train_positive_tensor),0)
y_train = torch.cat((y_train_negative_tensor,y_train_positive_tensor),0)

In [48]:
print("x_train: ",x_train.size())
print("y_train: ",y_train.size())

x_train:  torch.Size([53598, 2048])
y_train:  torch.Size([53598])


<h3>Test Negative

In [49]:
test_negative_path = r'Data/LSIFIR/Classification/Test/neg'
num_test_negative_img = 22050

In [50]:
test_negative_array = read_images(test_negative_path,num_test_negative_img)

In [52]:
x_test_negative_tensor = torch.tensor(test_negative_array[:20855,:])
y_test_negative_tensor = torch.zeros(20855, dtype= torch.long)

<h3>Test Positive

In [53]:
test_positive_path = r'Data/LSIFIR/Classification/Test/pos'
num_test_positive_img = 5944

In [54]:
test_positive_array = read_images(test_positive_path,num_test_positive_img)

In [55]:
x_test_positive_tensor = torch.tensor(test_positive_array)
y_test_positive_tensor = torch.ones(num_test_positive_img, dtype=torch.long)

<h3>Concat Test

In [57]:
x_test = torch.cat((x_test_negative_tensor, x_test_positive_tensor), 0)
y_test = torch.cat((y_test_negative_tensor, y_test_positive_tensor), 0)